In [1]:
import sys

sys.path.insert(0, "/Users/neugebauer/gitProjects/pyiron_core")

In [2]:
import pyiron_workflow.graph.graph_operations as go
from pyiron_workflow import as_function_node
from pyiron_workflow.graph import base, gui
from pyiron_workflow.simple_workflow import Workflow
import pyiron_nodes as pn
# from pyironflow.pyironflow import PyironFlow

2025-02-18 13:20:09,815 - pyiron_log - WARNING - pyiron found a 'dynamic' folder in the /Users/neugebauer/pyiron/resources resource directory. These are no longer supported in pyiron_base >=0.7.0. They are replaced by Project.create_job_class() and Project.wrap_python_function().
2025-02-18 13:20:09,816 - pyiron_log - WARNING - pyiron found a 'templates' folder in the /Users/neugebauer/pyiron/resources resource directory. These are no longer supported in pyiron_base >=0.7.0. They are replaced by Project.create_job_class() and Project.wrap_python_function().


In [3]:
graph = base.Graph('test')

node = pn.atomistic.structure.build.CubicBulkCell('Al')
graph += node

# base.GuiGraph(graph, full_graph=True, sleep=2.)
graph.nodes["CubicBulkCell"].graph.nodes["repeat"].node.inputs


Adding macro node CubicBulkCell
wf_label:  CubicBulkCell
target:  vacancy structure


,label,type,default,ready,value,node
0,structure,NonPrimitive,NotData,True,label ...,<pyiron_workflow.simple_workflow.Node object a...
1,repeat_scalar,int,1,True,label ...,<pyiron_workflow.simple_workflow.Node object a...


In [4]:
graph.nodes["CubicBulkCell"].graph.nodes

,id,import_path,label,parent_id,level,node,graph,node_type,widget_type,expanded
0,va_i_CubicBulkCell__element,pyiron_workflow.graph.base.identity,va_i_CubicBulkCell__element,CubicBulkCell,1,<pyiron_workflow.simple_workflow.Node object a...,None,node,customNode,False
1,va_i_CubicBulkCell__cell_size,pyiron_workflow.graph.base.identity,va_i_CubicBulkCell__cell_size,CubicBulkCell,1,<pyiron_workflow.simple_workflow.Node object a...,None,node,customNode,False
2,va_i_CubicBulkCell__vacancy_index,pyiron_workflow.graph.base.identity,va_i_CubicBulkCell__vacancy_index,CubicBulkCell,1,<pyiron_workflow.simple_workflow.Node object a...,None,node,customNode,False
3,bulk,pyiron_nodes.atomistic.structure.build.Bulk,bulk,CubicBulkCell,1,<pyiron_workflow.simple_workflow.Node object a...,None,node,customNode,False
4,repeat,pyiron_nodes.atomistic.structure.transform.Repeat,repeat,CubicBulkCell,1,<pyiron_workflow.simple_workflow.Node object a...,None,node,customNode,False
5,vacancy,pyiron_nodes.atomistic.structure.transform.Cre...,vacancy,CubicBulkCell,1,<pyiron_workflow.simple_workflow.Node object a...,None,node,customNode,False
6,va_o_CubicBulkCell__structure,pyiron_workflow.graph.base.identity,va_o_CubicBulkCell__structure,CubicBulkCell,1,<pyiron_workflow.simple_workflow.Node object a...,None,node,customNode,False


In [5]:
node.kwargs

{'element': 'Al', 'cell_size': 1, 'vacancy_index': None}

In [6]:
base.get_graph_from_macro_node(node).nodes["repeat"].node.inputs

wf_label:  CubicBulkCell
target:  vacancy structure


,label,type,default,ready,value,node
0,structure,NonPrimitive,NotData,True,label ...,<pyiron_workflow.simple_workflow.Node object a...
1,repeat_scalar,int,1,True,label ...,<pyiron_workflow.simple_workflow.Node object a...


Why not running the graph rather than the workflow. Graph is much more powerful and includes serialization and subgraphs? This works already! Does not yet have concept of input.

In [7]:
wf = Workflow('test')
wf.bulk = pn.atomistic.structure.build.Bulk('Al', cubic=True)
wf.repeat = pn.atomistic.structure.transform.Repeat(wf.bulk, repeat_scalar=1)
wf.repeat2 = pn.atomistic.structure.transform.Repeat(wf.repeat, repeat_scalar=1)
# wf.lammps = pn.atomistic.engine.lammps.Code1(structure=wf.repeat2)

full_graph = base.get_full_graph_from_wf(wf)
sub_graph = base._get_subgraph(full_graph, [1]) # collaps if node is macro node

# TODO: 
# - extend output option (include all child node outputs)
# - substitute selected nodes with ports having outside connections by a single virtual (graph-only) node
# - add parent group as graph node
# - make all inner nodes to child nodes of the parent group
# - create node object for the parent group

base.GuiGraph(full_graph, full_graph=True, height=400)
# node = base.graph_to_node(sub_graph)
# node
# node.outputs, sub_graph.label, 

ReactFlowWidget(layout=Layout(height='400px', width='800px'))

In [8]:
# TODO: 
# - extend output option (include all child node outputs)
# - substitute selected nodes with ports having outside connections by a single virtual (graph-only) node
# - add parent group as graph node
# - make all inner nodes to child nodes of the parent group
# - create node object for the parent group

In [9]:
full_graph.nodes;

In [10]:
node = base.graph_to_node(sub_graph)
node.inputs

,label,value,type,default,ready,node
0,repeat__structure,label ...,NonPrimitive,NotData,True,<pyiron_workflow.simple_workflow.Node object a...


In [11]:
base.get_externally_connected_input_ports(sub_graph)

[('repeat', 'structure')]

In [12]:
base.get_non_default_input(sub_graph)

{'repeat': {'structure': label                                            structure
  type                                     builtins.NoneType
  ready                                                False
  value                                                 None
  node     <pyiron_workflow.simple_workflow.Node object a...
  dtype: object}}

In [13]:
base.graph_to_node(sub_graph).outputs

,label,value,type,ready,node
0,repeat__structure,None,builtins.str,False,<pyiron_workflow.simple_workflow.Node object a...


In [14]:
new_graph = base.create_group(full_graph, [0,1], label="subgraph")
new_graph.edges

labels ['bulk__name', 'bulk__cubic']
GraphEdge(source='va_i_subgraph__bulk__name', target='bulk', sourceHandle='x', targetHandle='name')
GraphEdge(source='va_i_subgraph__bulk__cubic', target='bulk', sourceHandle='x', targetHandle='cubic')
labels ['repeat__structure']
['subgraph']


,source,target,sourceHandle,targetHandle
0,bulk,repeat,structure,structure
1,va_o_subgraph__repeat__structure,repeat2,x,structure
2,va_i_subgraph__bulk__name,bulk,x,name
3,va_i_subgraph__bulk__cubic,bulk,x,cubic
4,repeat,va_o_subgraph__repeat__structure,structure,x


Remark: Make sure that all operations regarding virtual nodes and edges are encapsulated to not affect any other groups

In [15]:
wf = Workflow("test")
wf.bulk = pn.atomistic.structure.build.Bulk("Al", cubic=True)
wf.repeat = pn.atomistic.structure.transform.Repeat(wf.bulk, repeat_scalar=1)
wf.repeat2 = pn.atomistic.structure.transform.Repeat(wf.repeat, repeat_scalar=1)
wf.lammps = pn.atomistic.engine.lammps.Code1(structure=wf.repeat2)

full_graph = base.get_full_graph_from_wf(wf)

new_graph = base.create_group(full_graph, [0, 1, 2], label="subgraph")

new_graph.nodes["subgraph"].expanded = True
new_graph.nodes["lammps"].expanded = False

# True, True gives error

new_graph = base.get_updated_graph(new_graph)

base.GuiGraph(new_graph, full_graph=True, height=400)

Adding macro node lammps
wf_label:  lammps
target:  Collect generic
target:  InitLammps path
Expanding node lammps
labels ['bulk__name', 'bulk__cubic']
GraphEdge(source='va_i_subgraph__bulk__name', target='bulk', sourceHandle='x', targetHandle='name')
GraphEdge(source='va_i_subgraph__bulk__cubic', target='bulk', sourceHandle='x', targetHandle='cubic')
labels ['repeat2__structure']
['subgraph']
Expanding node subgraph


ReactFlowWidget(layout=Layout(height='400px', width='800px'))

In [16]:
g = base._remove_virtual_nodes(new_graph, parent_label='lammps')
g = base._remove_virtual_nodes(g, parent_label='subgraph')
g = base.remove_hidden_nodes(g, "lammps")
g.nodes

,id,import_path,label,parent_id,level,node,graph,node_type,widget_type,expanded
0,subgraph,None,subgraph,None,0,<pyiron_workflow.simple_workflow.Node object a...,"Graph(id=None, label='subgraph', root_node=Non...",graph,customNode,True
1,bulk,pyiron_nodes.atomistic.structure.build.Bulk,bulk,subgraph,1,<pyiron_workflow.simple_workflow.Node object a...,None,node,customNode,False
2,repeat,pyiron_nodes.atomistic.structure.transform.Repeat,repeat,subgraph,1,<pyiron_workflow.simple_workflow.Node object a...,None,node,customNode,False
3,repeat2,pyiron_nodes.atomistic.structure.transform.Repeat,repeat2,subgraph,1,<pyiron_workflow.simple_workflow.Node object a...,None,node,customNode,False
4,lammps,pyiron_nodes.atomistic.engine.lammps.Code1,lammps,None,0,<pyiron_workflow.simple_workflow.Node object a...,"Graph(id='lammps', label='lammps', root_node=N...",graph,customNode,False


In [17]:
new_graph.nodes

,id,import_path,label,parent_id,level,node,graph,node_type,widget_type,expanded
0,subgraph,None,subgraph,None,0,<pyiron_workflow.simple_workflow.Node object a...,"Graph(id=None, label='subgraph', root_node=Non...",graph,customNode,True
1,bulk,pyiron_nodes.atomistic.structure.build.Bulk,bulk,subgraph,1,<pyiron_workflow.simple_workflow.Node object a...,None,node,customNode,False
2,repeat,pyiron_nodes.atomistic.structure.transform.Repeat,repeat,subgraph,1,<pyiron_workflow.simple_workflow.Node object a...,None,node,customNode,False
3,repeat2,pyiron_nodes.atomistic.structure.transform.Repeat,repeat2,subgraph,1,<pyiron_workflow.simple_workflow.Node object a...,None,node,customNode,False
4,lammps,pyiron_nodes.atomistic.engine.lammps.Code1,lammps,None,0,<pyiron_workflow.simple_workflow.Node object a...,"Graph(id='lammps', label='lammps', root_node=N...",graph,customNode,False


In [18]:
new_graph.edges

,source,target,sourceHandle,targetHandle
0,bulk,repeat,structure,structure
1,repeat,repeat2,structure,structure
2,bulk,repeat,structure,structure
3,repeat,repeat2,structure,structure
4,repeat2,lammps,structure,structure


In [19]:
new_graph.__getstate__()['nodes']['subgraph']['graph']

{'label': 'subgraph',
 'nodes': {'bulk': {'id': 'bulk',
   'import_path': 'pyiron_nodes.atomistic.structure.build.Bulk',
   'label': 'bulk',
   'parent_id': None,
   'level': 0,
   'graph': None,
   'node_type': 'node',
   'widget_type': 'customNode',
   'expanded': False,
   'node': {'label': 'bulk',
    'function': 'pyiron_nodes.atomistic.structure.build.Bulk',
    'inputs': {'name': 'Al', 'cubic': True}}},
  'repeat': {'id': 'repeat',
   'import_path': 'pyiron_nodes.atomistic.structure.transform.Repeat',
   'label': 'repeat',
   'parent_id': None,
   'level': 0,
   'graph': None,
   'node_type': 'node',
   'widget_type': 'customNode',
   'expanded': False,
   'node': {'label': 'repeat',
    'function': 'pyiron_nodes.atomistic.structure.transform.Repeat',
    'inputs': {}}},
  'repeat2': {'id': 'repeat2',
   'import_path': 'pyiron_nodes.atomistic.structure.transform.Repeat',
   'label': 'repeat2',
   'parent_id': None,
   'level': 0,
   'graph': None,
   'node_type': 'node',
   'widg

In [20]:
new_graph.__getstate__()['nodes']['lammps']['node']['inputs']

{'calculator': 'NotData'}

In [21]:
# base._save_graph(new_graph2, "new_graph2.json")
loaded_graph = base._load_graph("new_graph2.json")

FileNotFoundError: File 'new_graph2.json' not found in dir ..

In [ ]:
base.display_gui_data(new_graph2)


In [ ]:
new_graph = base.create_group(full_graph, [0,1], label="subgraph")
new_graph = base.move_parent_nodes_to_top(new_graph)
new_graph.nodes["subgraph"].expanded = True
new_graph = base.get_updated_graph(new_graph)

# new_graph0 = base.collapse_node(new_graph, "subgraph")
# new_graph = base.expand_node(new_graph, "subgraph")
# new_graph1 = base._remove_virtual_edges(new_graph0)
# new_graph2 = base._remove_edges_to_hidden_nodes(new_graph1)
# new_graph = base._remove_virtual_nodes(new_graph, reconnect_edges=True, parent_label="subgraph")
# new_graph = base._remove_virtual_nodes(new_graph, reconnect_edges=True, parent_label="lammps")
# new_graph = base.update_execution_graph(new_graph)

new_graph.edges

In [ ]:
new_graph0.edges

TODO:
- remove duplicate edges

In [ ]:
xx

In [22]:
from pyiron_workflow.graph import gui

pf = gui.PyironFlow([wf]) # 'flow_macro', 'lammps_test'])
pf.gui

Adding macro node lammps
wf_label:  lammps
target:  Collect generic
target:  InitLammps path
Expanding node lammps


Expanding node lammps
Expanding node lammps
Expanding node lammps
Expanding node lammps


Creating new tab for Workflow_1 1 1


In [47]:
base.graph_to_code(pf.graph)

Expanding node lammps


StopIteration: 

In [43]:
import inspect
print(inspect.getsource(macro_node._func))

@as_macro_node(labels=["generic", "path"])
def Code1(
    structure: Atoms,
    calculator,  # =InputCalcStatic(),  # TODO: Don't use mutable defaults
    potential: Optional[str] = None,
    working_dir: str = "test2",
):

    from pyiron_workflow import Workflow

    wf = Workflow("LammpsMacro")

    wf.Potential = Potential(structure=structure, name=potential)
    # wf.DummyNode = DummyNode(structure1=structure, structure2=structure)    

    wf.ListPotentials = ListPotentials(structure=structure)

    # wf.calc = CalcMD(calculator)

    wf.InitLammps = InitLammps(
        structure=structure,
        potential=wf.Potential,
        # calculator=wf.calc,
        calculator=calculator,
        working_directory=working_dir,
    )

    wf.Shell = Shell(
        # command=ExecutablePathResolver(module="lammps", code="lammps").path(),
        working_directory=wf.InitLammps,
    )

    wf.ParseLogFile = ParseLogFile(log_file=wf.Shell.outputs.log)
    wf.ParseDumpFile = ParseDumpFile(dum

In [45]:
macro_node = pf.graph.nodes # ['lammps'].node
# macro_node._func(**macro_node.kwargs)

In [35]:
pf.graph.nodes['lammps'].node.run() #inputs.structure.value

AttributeError: 'NoneType' object has no attribute 'get_chemical_symbols'

In [28]:
pf.graph.nodes['lammps'].node.run()

AttributeError: 'NoneType' object has no attribute 'get_chemical_symbols'

In [ ]:
base._graph_to_gui(sub_graph)

In [ ]:
def Subgraph(structure, ):

    from pyiron_workflow import Workflow
    import pyiron_nodes

    wf = Workflow('subgraph')

    wf.Repeat = pyiron_nodes.atomistic.structure.transform.Repeat(structure=structure) 
    wf.Repeat_2 = pyiron_nodes.atomistic.structure.transform.Repeat(structure=wf.Repeat) 
    wf.Plot3d = pyiron_nodes.atomistic.structure.view.Plot3d(structure=structure) 

    return wf.Repeat_2

structure = pn.atomistic.structure.build.Bulk("Al", cubic=True).run()
Subgraph(structure=structure).pull()

In [ ]:
def my_func(structure=None, repeat_scalar=1):
    from pyiron_workflow import Workflow
    import pyiron_nodes

    wf = Workflow("subgraph")

    # wf.va_i_subgraph__structure = __main__.InputNode()
    wf.Repeat = pyiron_nodes.atomistic.structure.transform.Repeat(structure=structure, repeat_scalar=repeat_scalar)
    wf.Repeat_2 = pyiron_nodes.atomistic.structure.transform.Repeat(structure=wf.Repeat)
    wf.Plot3d = pyiron_nodes.atomistic.structure.view.Plot3d(structure=structure)

    return wf.Repeat_2.pull()
   
structure = pn.atomistic.structure.build.Bulk("Al", cubic=True).run()
my_func(structure=structure, repeat_scalar=2);


In [ ]:
structure = pn.atomistic.structure.build.Bulk("Al", cubic=True).run()
import pyiron_nodes

wf = Workflow("subgraph")

# wf.va_i_subgraph__structure = __main__.InputNode()
wf.Repeat = pyiron_nodes.atomistic.structure.transform.Repeat(structure=structure)
wf.Repeat_2 = pyiron_nodes.atomistic.structure.transform.Repeat(structure=wf.Repeat, repeat_scalar=1)
wf.Plot3d = pyiron_nodes.atomistic.structure.view.Plot3d(structure=structure)

wf.run()


## From Graph

In [ ]:
import pyiron_nodes as pn


@as_function_node
def InputNode(structure=None):
    return structure


@as_function_node
def OutputNode(structure=None):
    return structure

InputStructure = "va_i_subgraph__structure"
OutputStructure = "va_o_subgraph__structure"

sub_graph = base.Graph(label="subgraph")
sub_graph += InputNode(label=InputStructure)
#sub_graph += pn.atomistic.structure.transform.Repeat(label="Repeat")
#sub_graph += pn.atomistic.structure.transform.Repeat(label="Repeat_2")
#sub_graph += pn.atomistic.structure.view.Plot3d(label="Plot3d")
sub_graph += OutputNode(label=OutputStructure)

#sub_graph += base.GraphEdge("Repeat", "Repeat_2", "structure", "structure")
#sub_graph += base.GraphEdge(InputStructure, "Repeat", "structure", "structure")
#sub_graph += base.GraphEdge(InputStructure, "Plot3d", "structure", "structure")
#sub_graph += base.GraphEdge("Repeat_2", OutputStructure, "structure", "structure")

graph = base.Graph(label="Workflow")
graph += pn.atomistic.structure.build.Bulk(name="Al", label="structure")
graph += pn.atomistic.structure.transform.Repeat(label="repeat")
graph += sub_graph
graph.nodes["subgraph"].expanded = False
graph.nodes["subgraph"].node = base.graph_to_node(sub_graph)

graph += pn.atomistic.structure.view.Plot3d(label="plot")

graph += base.GraphEdge("structure", "repeat", "structure", "structure")
graph += base.GraphEdge(OutputStructure, "plot", "structure", "structure")
graph += base.GraphEdge("repeat", InputStructure, "structure", "structure")
# graph += base.GraphEdge("subgraph", "plot", "structure", "structure")
# graph += base.GraphEdge("repeat", "subgraph", "structure", "structure")

graph = base.remove_node(graph, "Repeat")
graph = base.remove_node(graph, "Repeat_2")


base.GuiGraph(graph);

In [ ]:
sub_graph = base.Graph(label="subgraph")
sub_graph += pn.atomistic.structure.build.Bulk(label='Bulk', name='Al')
sub_graph += pn.atomistic.structure.transform.Repeat(label="Repeat", repeat_scalar=2)
sub_graph += pn.atomistic.structure.transform.Repeat(label="Repeat_2")
sub_graph += pn.atomistic.structure.view.Plot3d(label="Plot3d")
# sub_graph += OutputNode(label=OutputStructure)

sub_graph += base.GraphEdge("Repeat", "Repeat_2", "structure", "structure")
sub_graph += base.GraphEdge("Bulk", "Repeat", "structure", "structure")
sub_graph += base.GraphEdge("Bulk", "Plot3d", "structure", "structure")
# sub_graph += base.GraphEdge("Repeat_2", OutputStructure, "structure", "structure")

# sub_graph = base.update_execution_graph(sub_graph)

graph = base.Graph(label="Workflow")
graph += pn.atomistic.structure.build.Bulk(name="Al", label="structure")
graph += pn.atomistic.structure.transform.Repeat(label="repeat")
# graph += base.graph_to_node(sub_graph)
graph += sub_graph 
graph.nodes["subgraph"].expanded = False
graph.nodes["subgraph"].node = base.graph_to_node(sub_graph)

graph += pn.atomistic.structure.view.Plot3d(label="plot")

graph += base.GraphEdge("structure", "repeat", "structure", "structure")
# graph += base.GraphEdge(OutputStructure, "plot", "structure", "structure")
# graph += base.GraphEdge("repeat", InputStructure, "structure", "structure")

base.get_inputs_of_graph(sub_graph, exclude_unconnected_default_ports=True)
base.GuiGraph(graph)

In [ ]:
node = base.graph_to_node(sub_graph)
node.inputs.repeat_scalar = 1
node.inputs

node.run()

In [ ]:
graph.nodes["subgraph"].expanded = True
graph.nodes

In [ ]:
xx

In [ ]:
print(graph.nodes["subgraph"].expanded)
# Collapse a node
collapsed_graph = base.collapse_node(graph, "subgraph")

# Expand a node
expanded_graph = base._expand_node(graph, "subgraph")
print(graph.nodes["subgraph"].expanded)


In [ ]:
graph.label

In [ ]:
pf = gui.PyironFlow([graph])
pf.gui

In [ ]:
base.GuiGraph(collapsed_graph);

In [ ]:
graph.nodes["subgraph"].node.__getstate__() # correct function has to be constructed and stored in the node!

## From workflow

In [ ]:
wf = Workflow('lammps_full')

# wf.structure = pn.atomistic.structure.build.Bulk('Al', cubic=True)
wf.lammps = pn.atomistic.engine.lammps.Code1() # structure=wf.structure)
# wf.md_output = pn.atomistic.calculator.data.OutputCalcMD(dataclass=wf.lammps.outputs.generic)

In [ ]:
graph = base.Graph(label='test')
graph += pn.atomistic.engine.lammps.Code1()
graph.nodes["Code1"].expanded = False

base.GuiGraph(graph)
                   

In [ ]:
# graph.nodes['Code1'].graph

In [ ]:
graph = base.get_full_graph_from_wf(wf)


In [ ]:
base.get_import_path_from_type(pn.atomistic.engine.lammps.Code1)

In [ ]:
# wf = Workflow("flow")
# wf.LoadGraph = pn.graphs.flow.LoadGraph('lammps_full')
# wf.DisplayNodes = pn.graphs.flow.DisplayNodes(graph=wf.LoadGraph)
# wf.DisplayNodes.pull()

# graph = base.get_full_graph_from_wf(wf)

In [ ]:
# base.GuiGraph(pf.graph)

In [ ]:
pf = gui.PyironFlow([wf, 'flow_expand', 'edges'])
pf.gui

In [ ]:
from pyiron_workflow import as_macro_node

In [ ]:
graph = base.Graph('test')
graph += pf.graph.nodes['Repeat']

pf.graph.nodes['Repeat'].node.outputs.structure

In [ ]:
base.get_inputs_of_graph(pf.graph, exclude_unconnected_default_ports=True) 

In [ ]:
base.get_inputs_of_graph(graph, exclude_unconnected_default_ports=False)

In [ ]:
base.get_outputs_of_graph(pf.graph)

In [ ]:
print(base.graph_to_code(pf.graph))

In [ ]:
node = base.graph_to_node(pf.graph, exclude_unconnected_default_ports=False)
node.inputs.repeat_scalar = 1
node.run()
node.inputs

In [ ]:
import inspect

inspect.getsource(node._func)

In [ ]:
import types
from functools import partial

def graph_to_code(graph):
    graph = base.get_updated_graph(graph)
    graph = base.topological_sort(graph)
    graph = base.get_code_from_graph(graph)
    return graph


def graph_to_node(graph: base.Graph, exclude_unconnected_default_ports=True) -> base.Node:
    
    function_string = graph_to_code(graph)
    
    # Create a dictionary to serve as the local namespace
    virtual_namespace = {}
    
    # Execute the function string in the local namespace
    exec(function_string, globals(), virtual_namespace)
    
    # Retrieve the function from the local namespace
    func = virtual_namespace[graph.label]

    node = base.Node(
        func=func,
        label=graph.label,
        node_type="graph",
        inputs=base.get_inputs_of_graph(graph, exclude_unconnected_default_ports=True),
        outputs=base.get_outputs_of_graph(graph),
    )
    node.label = graph.label  # should not be necessary


    def _run(node):
        port = func(**node.kwargs)

        return port.node._workflow.run()
    
    node._run = types.MethodType(_run, node)

    return node



node = graph_to_node(pf.graph)
node.inputs.repeat_scalar = 1

node.run()
node.outputs.structure.value

In [ ]:


function_string = graph_to_code(pf.graph)

# Create a dictionary to serve as the local namespace
virtual_namespace = {}

# Execute the function string in the local namespace
exec(function_string, globals(), virtual_namespace)

# Retrieve the function from the local namespace
func = virtual_namespace['Workflow_3']

port = func(name='Fe', repeat_scalar=2)
port.node._workflow.run()

In [ ]:
# Define the function as a string
function_string = """
def Workflow_3(name: str, cubic: bool = False, repeat_scalar: int = 1):

    from pyiron_workflow import Workflow
    import pyiron_nodes

    wf = Workflow('Workflow_3')

    wf.Bulk = pyiron_nodes.atomistic.structure.build.Bulk(name=name, cubic=cubic) 
    wf.Repeat = pyiron_nodes.atomistic.structure.transform.Repeat(structure=wf.Bulk, repeat_scalar=repeat_scalar) 

    return wf.Repeat.outputs.structure
"""

# Create a dictionary to serve as the local namespace
virtual_namespace = {}

# Execute the function string in the local namespace
exec(function_string, globals(), virtual_namespace)

# Retrieve the function from the local namespace
func = virtual_namespace['Workflow_3']

port = func(name='Fe', repeat_scalar=2)
port.node._workflow.run()

In [ ]:
port.label

In [ ]:
virtual_namespace

In [ ]:
xx

In [ ]:
label = 'lammps'
print(graph.nodes[label].expanded)
# Collapse a node
collapsed_graph = base.collapse_node(graph, label)
print(graph.nodes[label].expanded)

# Expand a node
expanded_graph = base.expand_node(graph, label)
print(graph.nodes[label].expanded)

In [ ]:
collapsed_graph.edges

In [ ]:
base.GuiGraph(collapsed_graph)

In [ ]:
base.GuiGraph(expanded_graph)

In [ ]:
base.GuiGraph(graph)

In [ ]:
graph.nodes

In [ ]:
collapsed_graph.nodes

In [ ]:
graph.nodes['lammps'].expanded = False

u_graph = base.get_updated_graph(graph)

graph.nodes['lammps'].expanded 



In [ ]:
base.GuiGraph(u_graph)